# CW Laser Tuning Technique

In [1]:
import Ring_core as rc
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import pickle

%load_ext autoreload
%autoreload 2

### Microresonator parameters

In [2]:
ring_parameters =   {  
                        'N': 511, # Number of modes. It must be odd!
                        'n0': 2.4, # Refractive index
                        'n2': 2.4e-19, # Nonlinear reftactive index [m^2/W]
                        'FSR': 100e9, # Free Spectral Range [Hz]
                        'lambda0': 1553.4e-9, # CW pump wavelength [m]
                        'kappa': 3e8, # Optical linewidth [Hz]
                        'eta': 0.5, # Coupling efficiency
                        'Veff': 1e-15, # Effective mode volume [m^3]
                        'D2': 2.5e6, # Second order dispersion [Hz]
                        'Pin': 2 # Pump power [W]
                    }

## Ring 1 - Forward tuning with no effects (Multi-Soliton generation)

In [3]:
ring1 = rc.Ring(ring_parameters) # Init Ring class

### Simulation parameters

In [4]:
simulation_options_ring1 =  { 
                                'Effects': None, # None or "Thermal" or "Avoided mode crossings"
                                'Noise': False # True or False (White noise)
                            }

forward_parameters_ring1 =  {
                                'dseta_start': -10, # Normalized detuning start
                                'dseta_end': 45, # Normalized detuning end
                                'dseta_step': 0.01, # Tuning step
                                'roundtrips_step': 10, # Roundtrips per tuning step
                                'Amu0': np.random.randn(ring_parameters['N']) + np.random.randn(ring_parameters['N']) * 1j, # Initial field
                            }

### Numerical simulation

In [5]:
dseta_forward_ring1, amu_forward_ring1, _ = ring1.numerical_simulation(forward_parameters_ring1, simulation_options_ring1) # Run forward simulation

Forward tuning: 


100%|██████████| 5500/5500 [03:50<00:00, 23.89it/s]


### Plot results

In [6]:
interact(lambda dseta_snap: ring1.plot_results(dseta_forward_ring1, amu_forward_ring1, dseta_snap=dseta_snap), 
         dseta_snap=widgets.FloatSlider(min=forward_parameters_ring1['dseta_start'], max=forward_parameters_ring1['dseta_end'], step=forward_parameters_ring1['dseta_step'], value=0, continuous_update=False)); # Plot results from simulation

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='dseta_snap', max=45.0, min=…

### Save data

In [ ]:
data_ring1 =    {
                    'ring_parameters': ring_parameters,
                    'simulation_options': simulation_options_ring1,
                    'forward_parameters': forward_parameters_ring1,
                    'dseta_forward': dseta_forward_ring1,
                    'amu_forward': amu_forward_ring1
                }

In [ ]:
with open('data_ring1.pickle', 'wb') as f:
    pickle.dump(data_ring1, f, pickle.HIGHEST_PROTOCOL)

## Ring 2 - Forward and backward tuning with thermal effects (Soliton switching)

In [7]:
ring2 = rc.Ring(ring_parameters) # Init Ring class

### Simulation parameters

In [17]:
simulation_options_ring2 =  { 
                                'Effects': 'Thermal', # None or "Thermal" or "Avoided mode crossings"
                                'Noise': False # True or False (White noise)
                            }

forward_parameters_ring2 =  {
                                'dseta_start': -10, # Normalized detuning start
                                'dseta_end': 65, # Normalized detuning end
                                'dseta_step': 0.01, # Tuning step
                                'roundtrips_step': 10, # Roundtrips per tuning step
                                'Amu0': np.random.randn(ring_parameters['N']) + np.random.randn(ring_parameters['N']) * 1j, # Initial field
                                'theta0': 0, # Initial normalized variation of temperature
                                'tauT': 1e-8, # Thermal relaxation time [s]
                                'n2T': 2.4e-18 # Coefficient of thermal nonlinearity [m^2/W]
                            }

backward_parameters_ring2 =  {
                                'dseta_start': 65, # Normalized detuning start
                                'dseta_end': 10, # Normalized detuning end
                                'dseta_step': -0.001, # Tuning step
                                'roundtrips_step': 20, # Roundtrips per tuning step
                                'tauT': 1e-12, # Thermal relaxation time [s]
                                'n2T': 2.4e-18 # Coefficient of thermal nonlinearity [m^2/W]
                            }

### Numerical simulation

In [18]:
dseta_forward_ring2, amu_forward_ring2, theta_forward_ring2 = ring2.numerical_simulation(forward_parameters_ring2, simulation_options_ring2) # Run forward simulation

Forward tuning: 


100%|██████████| 7500/7500 [04:40<00:00, 26.74it/s]


In [20]:
dseta_backward_ring2, amu_backward_ring2, theta_backward_ring2 = ring2.numerical_simulation(backward_parameters_ring2, simulation_options_ring2, dseta_forward_ring2, amu_forward_ring2, theta_forward_ring2) # Run backward simulation

Backward tuning: 


100%|██████████| 55000/55000 [1:19:04<00:00, 11.59it/s]     


### Plot results

In [21]:
interact(lambda dseta_snap: ring2.plot_results(dseta_forward_ring2, amu_forward_ring2, dseta_backward_ring2, amu_backward_ring2, dseta_snap=dseta_snap), 
         dseta_snap=widgets.FloatSlider(min=forward_parameters_ring2['dseta_start'], max=forward_parameters_ring2['dseta_end'], step=forward_parameters_ring2['dseta_step'], value=0, continuous_update=False)); # Plot results from simulation

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='dseta_snap', max=65.0, min=…

### Save data

In [ ]:
data_ring2 =    {
                    'ring_parameters': ring_parameters,
                    'simulation_options': simulation_options_ring2,
                    'forward_parameters': forward_parameters_ring2,
                    'backward_parameters': backward_parameters_ring2,
                    'dseta_forward': dseta_forward_ring2,
                    'amu_forward': amu_forward_ring2,
                    'theta_forward': theta_forward_ring2,
                    'dseta_backrward': dseta_backward_ring2,
                    'amu_backward': amu_backward_ring2,
                    'theta_backward': theta_backward_ring2,
                }

In [ ]:
with open('data_ring2.pickle', 'wb') as f:
    pickle.dump(data_ring2, f, pickle.HIGHEST_PROTOCOL)

## Ring 3 - Forward tuning with avoided mode crossings (Soliton crystal generation)

In [22]:
ring_parameters['Pin'] = 0.5 # Update Pin 
ring3 = rc.Ring(ring_parameters) # Init Ring class

### Simulation parameters

In [28]:
simulation_options_ring3 =  { 
                                'Effects': 'Avoided mode crossings', # None or "Thermal" or "Avoided mode crossings"
                                'Noise': False # True or False (White noise)
                            }

forward_parameters_ring3 =  {
                                'dseta_start': -10, # Normalized detuning start
                                'dseta_end': 12.5, # Normalized detuning end
                                'dseta_step': 0.01, # Tuning step
                                'roundtrips_step': 50, # Roundtrips per tuning step
                                'Amu0': np.random.randn(ring_parameters['N']) + np.random.randn(ring_parameters['N']) * 1j, # Initial field
                                'mode_perturbated': 15 # Position of the modal crossing
                            }

### Numerical simulation

In [30]:
dseta_forward_ring3, amu_forward_ring3, _ = ring3.numerical_simulation(forward_parameters_ring3, simulation_options_ring3) # Run forward simulation

Forward tuning: 


100%|██████████| 2250/2250 [07:24<00:00,  5.07it/s]


### Plot results

In [31]:
interact(lambda dseta_snap: ring3.plot_results(dseta_forward_ring3, amu_forward_ring3, dseta_snap=dseta_snap), 
         dseta_snap=widgets.FloatSlider(min=forward_parameters_ring3['dseta_start'], max=forward_parameters_ring3['dseta_end'], step=forward_parameters_ring3['dseta_step'], value=0, continuous_update=False)); # Plot results from simulation

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='dseta_snap', max=12.5, min=…

### Save data

In [ ]:
data_ring3 =    {
                    'ring_parameters': ring_parameters,
                    'simulation_options': simulation_options_ring3,
                    'forward_parameters': forward_parameters_ring3,
                    'dseta_forward': dseta_forward_ring3,
                    'amu_forward': amu_forward_ring3,
                }

In [ ]:
with open('data_ring3.pickle', 'wb') as f:
    pickle.dump(data_ring3, f, pickle.HIGHEST_PROTOCOL)

## References

- https://www.nature.com/articles/nphys3893
- https://infoscience.epfl.ch/record/188349
- https://www.nature.com/articles/s41567-019-0635-0
- https://onlinelibrary.wiley.com/doi/full/10.1002/lpor.201600276
- https://zenodo.org/record/3726531#.X2QMoGhKhPY